In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

LABELS = ['pocket','swing','texting','whatever'] 

### read raw data files and add label colomn by parsing the file name 

In [2]:
def loadFile(root,file):
    print 'loading : ' , file 
    data=pd.read_csv(os.path.join(root,file))
    data['source']=file  
    for label in LABELS:
        if file.lower().startswith(label.lower()):
            # print 'add label : ', label
            data['label']=label
            return data
    data['label']=LABELS[-1]
    return data

In [3]:
def loadFiles(inputDir):
    print 'loading files from : ' , inputDir 
    return pd.concat([loadFile(inputDir,f) for f in os.listdir(inputDir) if f.lower().endswith('.csv')])  

In [4]:
rdf = loadFiles(r'../raw-data/utf8')

loading files from :  ../raw-data/utf8
loading :  SwingRightPhoneDownScBody.csv
loading :  swingLeftHandPhoneDownScBody.csv
loading :  TextingPointingDorwatd.csv
loading :  swing3.csv
loading :  Pocket2.csv
loading :  PocketLeftAidePhoneDownScBodyLoose.csv
loading :  Texting2.csv
loading :  SwingRight2.csv
loading :  swing4.csv
loading :  swing2.csv


In [5]:
rdf.head()

,time,gfx,gFy,gFz,wx,wy,wz,p,Bx,By,Bz,Unnamed: 11,source,label
0,0.006,0.0143,0.4008,0.5604,0.2233,-0.4407,-0.4187,978.6127,-28.4439,-5.0415,-30.7419,NaN,SwingRightPhoneDownScBody.csv,swing
1,0.011,0.0143,0.4008,0.5604,0.2233,-0.4407,-0.4187,978.5864,-28.4439,-5.0415,-30.7419,NaN,SwingRightPhoneDownScBody.csv,swing
2,0.021,0.0144,0.4479,0.8753,0.2233,-0.4407,-0.4187,978.5864,-28.4439,-5.0415,-30.7419,NaN,SwingRightPhoneDownScBody.csv,swing
3,0.045,0.0144,0.4479,0.8753,-0.0982,0.4523,-0.0352,978.5864,-28.4439,-5.0415,-30.7419,NaN,SwingRightPhoneDownScBody.csv,swing
4,0.045,0.0144,0.4479,0.8753,-0.0982,0.4523,-0.0352,978.5864,-28.4439,-5.0415,-30.7419,NaN,SwingRightPhoneDownScBody.csv,swing


In [6]:
rdf.dropna(axis=1,inplace=True)
rdf.info()

# rdf.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 78392 entries, 0 to 11818
Data columns (total 13 columns):
time      78392 non-null float64
gfx       78392 non-null float64
gFy       78392 non-null float64
gFz       78392 non-null float64
wx        78392 non-null float64
wy        78392 non-null float64
wz        78392 non-null float64
p         78392 non-null float64
Bx        78392 non-null float64
By        78392 non-null float64
Bz        78392 non-null float64
source    78392 non-null object
label     78392 non-null object
dtypes: float64(11), object(2)
memory usage: 8.4+ MB


In [7]:
rdf.head()

,time,gfx,gFy,gFz,wx,wy,wz,p,Bx,By,Bz,source,label
0,0.006,0.0143,0.4008,0.5604,0.2233,-0.4407,-0.4187,978.6127,-28.4439,-5.0415,-30.7419,SwingRightPhoneDownScBody.csv,swing
1,0.011,0.0143,0.4008,0.5604,0.2233,-0.4407,-0.4187,978.5864,-28.4439,-5.0415,-30.7419,SwingRightPhoneDownScBody.csv,swing
2,0.021,0.0144,0.4479,0.8753,0.2233,-0.4407,-0.4187,978.5864,-28.4439,-5.0415,-30.7419,SwingRightPhoneDownScBody.csv,swing
3,0.045,0.0144,0.4479,0.8753,-0.0982,0.4523,-0.0352,978.5864,-28.4439,-5.0415,-30.7419,SwingRightPhoneDownScBody.csv,swing
4,0.045,0.0144,0.4479,0.8753,-0.0982,0.4523,-0.0352,978.5864,-28.4439,-5.0415,-30.7419,SwingRightPhoneDownScBody.csv,swing
